# Step 1: Import Libraries

In [49]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import PorterStemmer

# Download NLTK stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Step 2: Load the Dataset

In [50]:
# Load the dataset
df = pd.read_csv(r"C:\Users\user\Desktop\spam.csv", encoding='latin-1') 

# Display the first few rows of the dataframe
print(df.head())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


# Step 3: Drop Unnecessary Columns and Rename Columns

In [51]:
# Drop unnecessary columns and rename columns
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
df = df.rename(columns={'v1': 'label', 'v2': 'message'})

# Display the first few rows to confirm changes
print(df.head())


  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


# Step 4: Convert Labels to Binary Values

In [52]:
# Convert labels to binary values: 'spam' -> 1, 'ham' -> 0
df['label'] = df['label'].map({'ham': 0, 'spam': 1})


# Step 5: Preprocess the Text Data

In [53]:
# Initialize the stemmer
stemmer = PorterStemmer()

# Enhanced preprocess text function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    # Tokenize text
    words = text.split()
    # Remove stopwords and apply stemming
    stop_words = set(stopwords.words('english'))
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['message'] = df['message'].apply(preprocess_text)

# Display the first few rows to confirm preprocessing
print(df.head())


   label                                            message
0      0  go jurong point crazi avail bugi n great world...
1      0                              ok lar joke wif u oni
2      1  free entri wkli comp win fa cup final tkt st m...
3      0                u dun say earli hor u c alreadi say
4      0          nah dont think goe usf live around though


# Step 6: Add Additional Features 

In [54]:
# Add a new feature for message length
df['message_length'] = df['message'].apply(len)

# Display the first few rows to confirm the new feature
print(df.head())


   label                                            message  message_length
0      0  go jurong point crazi avail bugi n great world...              76
1      0                              ok lar joke wif u oni              21
2      1  free entri wkli comp win fa cup final tkt st m...             103
3      0                u dun say earli hor u c alreadi say              35
4      0          nah dont think goe usf live around though              41


# Step 7: Feature Extraction

In [55]:
# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df['message'])

# Convert to DataFrame and ensure all column names are strings
X_text_df = pd.DataFrame(X_text.toarray(), columns=vectorizer.get_feature_names_out())
X_text_df.columns = X_text_df.columns.astype(str)

# Combine text features with message length
X = pd.concat([X_text_df, df['message_length'].reset_index(drop=True)], axis=1)
y = df['label']


# Step 8: Split the Data

In [56]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Step 9: Hyperparameter Tuning

In [57]:
# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0]  # Add more values if needed
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_


# Step 10: Evaluate the Model

In [58]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.979372197309417
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.94      0.91      0.92       150

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



# Step 11: Predict on New Data

In [59]:
# Function to predict if a new message is spam or ham
def predict_spam(text):
    text = preprocess_text(text)
    text_tfidf = vectorizer.transform([text])
    text_tfidf_df = pd.DataFrame(text_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
    text_tfidf_df.columns = text_tfidf_df.columns.astype(str)
    message_length = len(text)
    text_features = pd.concat([text_tfidf_df, pd.DataFrame([message_length], columns=['message_length'])], axis=1)
    prediction = best_model.predict(text_features)
    return 'spam' if prediction == 1 else 'ham'

# Example prediction           
new_message = "Congratulations! You've won a free ticket to the Bahamas. Call now!"
new_message_2 = "I have to inform you that tomorrow is holiday"
print(f'New message prediction: {predict_spam(new_message)}')
print(f'New message prediction: {predict_spam(new_message_2)}')

New message prediction: spam
New message prediction: ham


In [36]:
!pip install scikit-learn

In [37]:
!pip install numpy

In [38]:
!pip install nltk